<a href="https://colab.research.google.com/github/4060E058/2020-AI-/blob/master/202011_04_Chapter_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Wrangling: Join, Combine, and Reshape（數據加工：連接, 合並, 整形）
在很多應用中，數據通常散落在不同的文件或數據庫中，並不方便進行分析。這一章主要關註工具，能幫我們combine, join, rearrange數據。

譯者：本章中的一些操作名稱翻譯上容易混淆，為了清楚區別幾個名字的翻譯，這里對第八章中出現的一些操作做一個翻譯對應表。雖然用不同名稱指代，但主要是為了謀求一貫性，讓讀者容易區分。這些操作本身其實區別還是比較小的，不用糾結於翻譯名稱，記住英文即可。

##8.1中的操作名

join：連接

combine：合並

reshape：整形

##8.2中的操作名

###merge：歸並

###concatenate：串聯

##8.3中的操作名

###pivot：旋轉

###stack：堆疊

我在整個第八章對這些翻譯做了更新，其他章節可能沒有統一，如果有發現到不統一的翻譯，可以在issue里提出，也可以直接pull request

#**8.1 Hierarchical Indexing（分層索引）**

Hierarchical Indexing是pandas中一個重要的特性，能讓我們在一個軸（axis）上有多個index levels（索引層級）。它可以讓我們在低維格式下處理高維數據。這里給出一個簡單的例子，構建一個series，其index是a list of lists:

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])

In [3]:
data

a  1   -0.040562
   2   -0.070149
   3    1.035813
b  1    0.505416
   3    0.645682
c  1   -1.386724
   2   -0.024583
d  2    0.472005
   3    0.215773
dtype: float64

**其中我們看到的是把MultiIndex作為index(索引)的，美化過後series。**

In [4]:
data.index


MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

**對於這種分層索引對象，partial indexing（部分索引）也是能做到的，這種方法可以讓我們簡潔地選中數據的一部分：**



In [5]:
data['b']


1    0.505416
3    0.645682
dtype: float64

In [6]:
data['b': 'c']


b  1    0.505416
   3    0.645682
c  1   -1.386724
   2   -0.024583
dtype: float64

In [7]:
data.loc[['b', 'd']]


b  1    0.505416
   3    0.645682
d  2    0.472005
   3    0.215773
dtype: float64

**selection（選中）對於一個內部層級（inner level）也是可能的：**

In [8]:
data.loc[:, 2]


a   -0.070149
c   -0.024583
d    0.472005
dtype: float64

**分層索引的作用是改變數據的形狀，以及做一些基於組的操作（group-based）**

**比如做一個數據透視表（pivot table）。**

**例子，我們可以用unstack來把數據進行重新排列，產生一個DataFrame：**

In [9]:
data.unstack()


,1,2,3
a,-0.040562,-0.070149,1.035813
b,0.505416,NaN,0.645682
c,-1.386724,-0.024583,NaN
d,NaN,0.472005,0.215773


**相反的操作是stack:**

In [10]:
data.unstack().stack()


a  1   -0.040562
   2   -0.070149
   3    1.035813
b  1    0.505416
   3    0.645682
c  1   -1.386724
   2   -0.024583
d  2    0.472005
   3    0.215773
dtype: float64

之後的章節會對unstack和stack做更多介紹。

###對於dataframe，任何一個axis(軸)都可以有一個分層索引：

In [11]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

**每一層級都可以有一個名字（字元串或任何python對象）。如果有的話，這些會顯示在輸出中：**



In [12]:
frame.index.names = ['key1', 'key2']


In [13]:
frame.columns.names = ['state', 'color']


In [14]:
frame


state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

**這里我們要註意區分行標簽(row label)中索引的名字'state'和'color'。**

**如果想要選中部分列(partial column indexing)的話，可以選中一組列（groups of columns）:**

In [15]:
frame['Ohio']


color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

**MultiIndex能被同名函數創建，而且可以重復被使用；在DataFrame中給列創建層級名可以通過以下方式：**



In [16]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                      names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

#1 Reordering and Sorting Levels（重排序和層級排序）

###有時候我們需要在一個axis（軸）上按層級進行排序，或者在一個層級上，根據值來進行排序。swaplevel會取兩個層級編號或者名字，並返回一個層級改變後的新對象（數據本身並不會被改變）：

In [17]:
frame.swaplevel('key1', 'key2')


state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

**另一方面，sort_index則是在一個層級上，按數值進行排序。比如在交換層級的時候，通常也會使用sort_index，來讓結果按指示的層級進行排序：**



In [18]:
frame.sort_index(level=1)


state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [19]:
frame.sort_index(level='color') 
frame.sort_index(level='state') 
# 这两个语句都会报错

KeyError: ignored

**（按照我的理解，level指的是key1和key2，key1是level=0，key2是level=1。可以看到下面的结果和上面是一样的：）**



In [22]:
frame.sort_index(level='key2') 


state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [23]:
frame.swaplevel(0, 1).sort_index(level=0) # 把key1余key2交换后，按key2来排序


state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

**如果index是按詞典順序那種方式來排列的話（比如從外層到內層按a,b,c這樣的順序），在這種多層級的index對象上，數據選擇的效果會更好一些。這是我們調用sort_index(level=0) or sort_index()**

#2 Summary Statistics by Level (按層級來歸納統計數據)
####在DataFrame和Series中，一些描述和歸納統計數據都是有一個level選項的，這里我們可以指定在某個axis下，按某個level（層級）來匯總。比如上面的DataFrame，我們可以按 行 或 列的層級來進行匯總：

In [24]:
frame


state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [25]:
frame.sum(level='key2')


state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [26]:
frame.sum(level='color', axis=1)


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

#3 Indexing with a DataFrame’s columns（利用DataFrame的列來索引）
####把DataFrame里的一列或多列作為行索引（row index）是一件很常見的事；另外，我們可能還希望把行索引變為列。這里有一個例子：

In [27]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


**DataFrame的set_index會把列作為索引，並創建一個新的DataFrame：**



In [28]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

**默認刪除原先的列，當然我們也可以留著：**



In [29]:
frame.set_index(['c', 'd'], drop=False)


a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

**另一方面，reset_index的功能與set_index相反，它會把多層級索引變為列：**



In [30]:
frame2.reset_index()


,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1
